In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Convolutional Neural Network

### Importing the libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder

# Part 1 - Data Preprocessing

## Define transforms

In [ ]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to a fixed size
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize images
])

## Import Dataset

In [ ]:
dataset = ImageFolder(root='/content/drive/MyDrive/Colab Notebooks/Projects/Project 2/Minerals/Minerals_dataset', transform=transform)

## Split Dataset into Training and Test set

In [ ]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

## Create data loaders

In [ ]:
batch_size = 32  # Increase batch size for better GPU utilization
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

# Part 2 - Building the CNN

### Initialising the CNN

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 56 * 56, 512)
        self.fc2 = nn.Linear(512, 7)  # 7 classes for the minerals

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = CNN().to(device)  # Move model to GPU if available


In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


# Part 3 - Training the CNN

### Load Pretrained Model

In [ ]:
# Load pretrained ResNet model
model = models.resnet18(pretrained=True)
# Freeze all layers except the final fully connected layer
for param in model.parameters():
    param.requires_grad = False
# Replace the final fully connected layer with a new one for our classification task
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 7)  # 7 classes for the minerals

model = model.to(device)  # Move model to GPU if available

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Early stopping parameters
best_val_loss = float('inf')
patience = 5
counter = 0

## Training and Validation

In [ ]:
# Training loop
for epoch in range(25):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)  # Move data to GPU if available
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)  # Move data to GPU if available
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)

    val_loss /= len(val_dataset)
    print(f'Epoch {epoch+1}/{25}, Validation Loss: {val_loss:.4f}')

    # Check for early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered!")
            break

# Save the trained model
torch.save(model.state_dict(), 'mineral_classification_model.pth')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 104MB/s]


Epoch 1/25, Validation Loss: 1.0304
Epoch 2/25, Validation Loss: 0.9086
Epoch 3/25, Validation Loss: 0.8159
Epoch 4/25, Validation Loss: 0.7853
Epoch 5/25, Validation Loss: 0.7113
Epoch 6/25, Validation Loss: 0.6746
Epoch 7/25, Validation Loss: 0.6452
Epoch 8/25, Validation Loss: 0.6704
Epoch 9/25, Validation Loss: 0.6180
Epoch 10/25, Validation Loss: 0.6098
Epoch 11/25, Validation Loss: 0.6096
Epoch 12/25, Validation Loss: 0.5802
Epoch 13/25, Validation Loss: 0.5813
Epoch 14/25, Validation Loss: 0.5493
Epoch 15/25, Validation Loss: 0.5445
Epoch 16/25, Validation Loss: 0.5484
Epoch 17/25, Validation Loss: 0.5421
Epoch 18/25, Validation Loss: 0.5314
Epoch 19/25, Validation Loss: 0.5242
Epoch 20/25, Validation Loss: 0.5009
Epoch 21/25, Validation Loss: 0.5170
Epoch 22/25, Validation Loss: 0.5054
Epoch 23/25, Validation Loss: 0.4940
Epoch 24/25, Validation Loss: 0.5122
Epoch 25/25, Validation Loss: 0.4891
